# Modelagem de Produtividade Agrícola com Random Forest

Este notebook utiliza o arquivo `integrado_anual.csv` para treinar e avaliar um modelo RandomForestRegressor, prevendo a produtividade (ton/ha) a partir de variáveis climáticas e NDVI.


## 1. Importação dos dados
As variáveis disponíveis são:
- Ano (referência)
- Área Plantada (ha)
- Área Colhida (ha)
- Produção (ton)
- Produtividade (ton/ha) [alvo]
- Savitzky-Golay (NDVI médio anual)
- Precipitacao_total_mm
- Temp_media_C
- Umidade_media


In [ ]:
import pandas as pd
df = pd.read_csv('../dados_processados/integrado_anual.csv')
df

## 2. Treinamento e Validação (Leave-One-Out)
O modelo Random Forest é robusto para pequenas amostras e captura relações não-lineares.
A validação Leave-One-Out garante avaliação realista mesmo com poucos anos.


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import LeaveOneOut
import numpy as np

X = df[['Área Plantada (ha)', 'Área Colhida (ha)', 'Savitzky-Golay',
        'Precipitacao_total_mm', 'Temp_media_C', 'Umidade_media']]
y = df['Produtividade (ton/ha)']

loo = LeaveOneOut()
y_true, y_pred = [], []

for train_idx, test_idx in loo.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred.append(model.predict(X_test)[0])
    y_true.append(y_test.values[0])


## 3. Avaliação do Modelo
As métricas utilizadas são R², MAE e RMSE.


In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
r2 = r2_score(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False)
print(f'R²: {r2:.3f}')
print(f'MAE: {mae:.3f}')
print(f'RMSE: {rmse:.3f}')


## 4. Gráfico Real vs Predito
Visualização da performance do modelo ao longo dos anos.


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,5))
plt.plot(df['Ano'], y_true, label='Real', marker='o')
plt.plot(df['Ano'], y_pred, label='Predito', marker='s')
plt.xlabel('Ano')
plt.ylabel('Produtividade (ton/ha)')
plt.title('Produtividade Real vs Predita (Random Forest)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## 5. Importância das Variáveis
Quais variáveis mais influenciam a previsão?


In [ ]:
import seaborn as sns
model_final = RandomForestRegressor(n_estimators=100, random_state=42)
model_final.fit(X, y)
importancias = model_final.feature_importances_
features = X.columns
plt.figure(figsize=(7,4))
sns.barplot(x=importancias, y=features, palette='viridis')
plt.title('Importância das Variáveis - Random Forest')
plt.xlabel('Importância')
plt.tight_layout()
plt.show()


---
Este notebook é um insumo direto para a Etapa 3 do projeto: demonstra como usar Random Forest para prever produtividade agrícola, avaliar desempenho e interpretar resultados.
